In [5]:
#import libraries
import pandas as pd
import streamlit as st
import pandas as pd
import plotly.express as px
from typing import Tuple

In [4]:
#load data
def load_data(path: str):
    df = pd.read_csv(path,parse_dates=True, infer_datetime_format=True )
    df.rename(columns=lambda c: c.strip().replace(" ", "_"), inplace=True)
    df["Invoice_Date"] = pd.to_datetime(df["Invoice_Date"], dayfirst=True, errors="coerce")
    df["year"] = df["Invoice_Date"].dt.year
    df["month"] = df["Invoice_Date"].dt.to_period("M").astype(str)
    return df

df = load_data("Nike Dataset.csv")
df

,Invoice_Date,Product,Region,Retailer,Sales_Method,State,Price_per_Unit,Total_Sales,Units_Sold,year,month
0,2020-01-01,Men's Street Footwear,Northeast,Foot Locker,In-store,New York,50,6000,120,2020,2020-01
1,2020-01-02,Men's Athletic Footwear,Northeast,Foot Locker,In-store,New York,50,5000,100,2020,2020-01
2,2020-01-03,Women's Street Footwear,Northeast,Foot Locker,In-store,New York,40,4000,100,2020,2020-01
3,2020-01-04,Women's Athletic Footwear,Northeast,Foot Locker,In-store,New York,45,3825,85,2020,2020-01
4,2020-01-05,Men's Apparel,Northeast,Foot Locker,In-store,New York,60,5400,90,2020,2020-01
...,...,...,...,...,...,...,...,...,...,...,...
9355,2021-07-06,Women's Athletic Footwear,West,West Gear,Outlet,Idaho,38,60,16,2021,2021-07
9356,2021-04-05,Women's Street Footwear,West,West Gear,Outlet,Idaho,19,31,16,2021,2021-04
9357,2021-05-05,Women's Street Footwear,West,West Gear,Outlet,Idaho,18,33,18,2021,2021-05
9358,2021-06-04,Women's Street Footwear,West,West Gear,Outlet,Idaho,34,63,19,2021,2021-06


In [15]:
def calculate_kpis(df: pd.DataFrame):
    """
    Compute key performance indicators:
     - total_sales: sum of total_sales
     - total_transactions: number of records
     - avg_order_value: average total_sales per transaction
    """
    Total_Sales = df["Total_Sales"].sum()
    total_transactions = len(df)
    avg_order_value = df["Total_Sales"].mean()
    return Total_Sales, total_transactions, avg_order_value

calculate_kpis(df)

(8629275, 9360, 921.9310897435897)

In [20]:
def time_series_chart(df: pd.DataFrame):
    """Daily sales line chart."""
    df_ts = df.groupby("Invoice_Date")["Total_Sales"].sum().reset_index()
    return px.line(df_ts, x="Invoice_Date", y="Total_Sales", title="Daily Sales Trend")

def region_bar_chart(df: pd.DataFrame) :
    """Sales by region bar chart."""
    df_reg = df.groupby("Region")["Total_Sales"].sum().reset_index()
    return px.bar(df_reg, x="Region", y="Total_Sales", title="Sales by Region")

def category_pie_chart(df: pd.DataFrame):
    """Sales distribution by product category (here using 'product')."""
    df_cat = df.groupby("Product")["Total_Sales"].sum().reset_index()
    top10 = df_cat.nlargest(10, "Total_Sales")
    return px.pie(top10, names="Product", values="Total_Sales", title="Top 10 Products by Sales")

def sales_method_chart(df: pd.DataFrame):
    """Sales by sales method (e.g., In-store vs Online)."""
    df_sm = df.groupby("Sales_Method")["Total_Sales"].sum().reset_index()
    return px.bar(df_sm, x="Sales_Method", y="Total_Sales", title="Sales by Sales Method")

In [23]:
# --- Page Config ---
st.set_page_config(
    page_title="Nike US Sales Dashboard",
    layout="wide",
    initial_sidebar_state="expanded",
)
# --- Sidebar ---
#st.sidebar.image("assets/nike_logo.png", width=120)
st.sidebar.image("nike_logo.png", width=120)

st.sidebar.title("Filters")

DATA_PATH = "Nike Dataset.csv"
df = load_data(DATA_PATH)

years = st.sidebar.multiselect("year", sorted(df["year"].unique()), default=sorted(df["year"].unique()))
regions = st.sidebar.multiselect("Region", df["Region"].unique(), default=df["Region"].unique())
methods = st.sidebar.multiselect("Sales Method", df["Sales_Method"].unique(), default=df["Sales_Method"].unique())

filtered = df[(df["year"].isin(years)) & (df["Region"].isin(regions)) &(df["Sales_Method"].isin(methods))]

# --- KPIs ---
Total_Sales, total_transactions, avg_order_value = calculate_kpis(filtered)
st.title("🏀 Nike US Sales Dashboard (2020–2021)")

col1, col2, col3 = st.columns(3)
col1.metric("Total Sales", f"${Total_Sales:,.0f}")
col2.metric("Total Transactions", total_transactions)
col3.metric("Avg. Order Value", f"${avg_order_value:,.2f}")


# --- Charts ---
st.markdown("## 📈 Sales Over Time")
st.plotly_chart(time_series_chart(filtered), use_container_width=True)



st.markdown("## 🌎 Sales by Region")
st.plotly_chart(region_bar_chart(filtered), use_container_width=True)

st.markdown("## 📦 Top Products")
st.plotly_chart(category_pie_chart(filtered), use_container_width=True)

st.markdown("## 🛒 Sales Method Breakdown")
st.plotly_chart(sales_method_chart(filtered), use_container_width=True)

# --- Download Data ---
st.markdown("## 📥 Download Filtered Data")
csv = filtered.to_csv(index=False)
st.download_button("Download CSV", data=csv, file_name="nike_sales_filtered.csv")



2025-08-09 07:50:38.961 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:38.963 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:38.973 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:38.974 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:38.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:38.976 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:38.977 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:38.978 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2025-08-09 07:50:39.610 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:39.610 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:39.612 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:39.704 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:39.706 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:39.707 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:39.707 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-09 07:50:39.708 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

False

streamlit
pandas
plotly
openpyxl